In [109]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import datetime
import time
import pickle
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import json
import folium
import warnings

In [57]:
def datetime_to_str(dt, format):
    return datetime.datetime.strftime(dt, format)

def str_to_datetime(str_date, format):
    if len(str_date) == 8:
        str_date += '01.'
    return datetime.datetime.strptime(str_date, format)

In [58]:
five_ago = datetime.datetime.now() - relativedelta(years=1)
five_ago = datetime_to_str(five_ago, '%Y.%m.')
five_ago_d = str_to_datetime(five_ago, "%Y.%m.%d.")

print(five_ago)
print(five_ago_d)

2022.08.
2022-08-01 00:00:00


In [64]:
# driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.96)
Stacktrace:
#0 0x55eb1ceace23 <unknown>
#1 0x55eb1cbd57a7 <unknown>
#2 0x55eb1cbba340 <unknown>
#3 0x55eb1cbbb84f <unknown>
#4 0x55eb1cbbb7a4 <unknown>
#5 0x55eb1cbabef1 <unknown>
#6 0x55eb1cbaccc0 <unknown>
#7 0x55eb1cc3fc42 <unknown>
#8 0x55eb1cc34fd8 <unknown>
#9 0x55eb1cc08a7b <unknown>
#10 0x55eb1cc0981e <unknown>
#11 0x55eb1ce6e638 <unknown>
#12 0x55eb1ce72507 <unknown>
#13 0x55eb1ce7cc4c <unknown>
#14 0x55eb1ce73136 <unknown>
#15 0x55eb1ce419cf <unknown>
#16 0x55eb1ce96b98 <unknown>
#17 0x55eb1ce96d68 <unknown>
#18 0x55eb1cea5cb3 <unknown>
#19 0x7fc175494b43 <unknown>


In [65]:
options = webdriver.ChromeOptions()

prefs = {
    'download.default_directory':'/home/haneol/dev_ws/EDA/data/',
    'download.propt_for_download':False
}

options.add_experimental_option('prefs', prefs)

url = 'https://new.land.naver.com/complexes?ms=37.49764,127.065056,17&a=APT:ABYG:JGC:PRE&e=RETAIL'
driver = webdriver.Chrome(
    service=Service('../driver/chromedriver-linux64/chromedriver'),
    options=options
)
driver.get(url)

actions = ActionChains(driver)

In [92]:
# 검색 id='search_input'
search = driver.find_element(By.ID, 'search_input')
search.send_keys('대치동 은마')
search.send_keys(Keys.ENTER)
time.sleep(1)

# 실세/실거래가 클릭
complex_link = driver.find_elements(By.CLASS_NAME, 'complex_detail_link')
actions.click(complex_link[0]).perform()
detail_contents = driver.find_element(By.CLASS_NAME, 'detail_contents_inner ')
time.sleep(1)
detailTab2 = detail_contents.find_element(By.ID, 'detailTab2')
actions.click(detailTab2).perform()
time.sleep(1)

# # '평'으로 전환
# tab_area_unit = detail_contents.find_element(By.CLASS_NAME, 'tab_area_unit')
# actions.click(tab_area_unit).perform()
# time.sleep(1)

# 사이즈 = [101, 115]
room_size = detail_contents.find_element(By.CLASS_NAME, 'detail_sorting_width')
tabs = room_size.find_elements(By.CLASS_NAME, 'detail_sorting_tab')
price_dict = {'date_real':[], 'real_price':[], 'date_market' : [], 'market_lower':[], 'market_upper':[]}
eunma_real_estate = {}
for size_option in tabs:
    key1 = size_option.text
    eunma_real_estate[key1] = {}
    actions.click(size_option).perform()
    driver.implicitly_wait(3)
    time.sleep(5)
    
    # 매매/전세 선택
    div_list = detail_contents.find_element(By.CLASS_NAME, 'detail_sorting_tabs--underbar')
    buy_or_rental = div_list.find_elements(By.TAG_NAME, 'a')
    for buy_option in buy_or_rental[:2]:
        key2 = buy_option.text
        eunma_real_estate[key1][key2] = price_dict
        buy_option.click()
        time.sleep(5)
        driver.implicitly_wait(3)

        price_info = driver.find_element(By.CLASS_NAME, 'detail_box--chart')

        real_price = price_info.find_element(By.CLASS_NAME, 'detail_data_table.type_real')
        real_price_list = real_price.find_elements(By.TAG_NAME, 'tr')[1:]
        last_date = real_price_list[-1].text.split('\n')[0]
        last_date = str_to_datetime(last_date, "%Y.%m.%d.")
        detail_data_more = price_info.find_elements(By.CLASS_NAME, 'detail_data_more')
        real_price_more = detail_data_more[0]
        actions.move_to_element(real_price_more).perform()
        
        while last_date > five_ago_d:
            actions.move_to_element(real_price_more).perform()
            real_price_more.click()
            driver.implicitly_wait(3)

            real_price = price_info.find_element(By.CLASS_NAME, 'detail_data_table.type_real')
            real_price_list = real_price.find_elements(By.TAG_NAME, 'tr')[1:]
            last_date = real_price_list[-1].text.split('\n')[0]  # str
            last_date = str_to_datetime(last_date, "%Y.%m.%d.")  # datetime


        for each_date in real_price_list:
            date = each_date.text.split('\n')[0]
            price = each_date.text.split('\n')[1:]
            eunma_real_estate[key1][key2]['date_real'].extend([date for _ in range(len(price))])
            eunma_real_estate[key1][key2]['real_price'].extend(price)


        market_price = price_info.find_element(By.CLASS_NAME, 'detail_data_table.type_price')
        market_price_list = market_price.find_elements(By.TAG_NAME, 'tr')[1:]
        last_date = market_price_list[-1].text.split(' ')[0]
        last_date = str_to_datetime(last_date, "%Y.%m.%d.")
        detail_data_more = price_info.find_elements(By.CLASS_NAME, 'detail_data_more')
        market_price_more = detail_data_more[1]
        actions.move_to_element(market_price_more).perform()
        
        while last_date > five_ago_d:
            actions.move_to_element(market_price_more).perform()
            market_price_more.click()
            driver.implicitly_wait(3)

            market_price = price_info.find_element(By.CLASS_NAME, 'detail_data_table.type_price')
            market_price_list = market_price.find_elements(By.TAG_NAME, 'tr')[1:]
            last_date = market_price_list[-1].text.split(' ')[0]
            last_date = str_to_datetime(last_date, "%Y.%m.%d.")

        for each_date in market_price_list:
            date = each_date.text.split(' ')[0]
            lower = each_date.text.split(' ')[1]
            upper = each_date.text.split(' ')[2]

            eunma_real_estate[key1][key2]['date_market'].append(date)
            eunma_real_estate[key1][key2]['market_lower'].append(lower)
            eunma_real_estate[key1][key2]['market_upper'].append(upper)

        actions.move_to_element(buy_option).perform()
        time.sleep(3)
    
    actions.move_to_element(size_option).perform()
    time.sleep(3)
        

In [107]:
print('매매 계약월:', len(eunma_real_estate['101㎡']['매매']['date_real']))
print('매매가:', len(eunma_real_estate['101㎡']['매매']['real_price']))
print('매매 시세 기준일:', len(eunma_real_estate['101㎡']['매매']['date_market']))
print('매매 시세 하한가:', len(eunma_real_estate['101㎡']['매매']['market_lower']))
print('매매 시세 상한가:', len(eunma_real_estate['101㎡']['매매']['market_upper']))
print()
print('전세 계약월:', len(eunma_real_estate['101㎡']['전세']['date_real']))
print('전세가:', len(eunma_real_estate['101㎡']['전세']['real_price']))
print('전세 시세 기준일:', len(eunma_real_estate['101㎡']['전세']['date_market']))
print('전세 시세 하한가:', len(eunma_real_estate['101㎡']['전세']['market_lower']))
print('전세 시세 상한가:', len(eunma_real_estate['101㎡']['전세']['market_upper']))

매매 계약월: 650
매매가: 650
매매 시세 기준일: 224
매매 시세 하한가: 224
매매 시세 상한가: 224

전세 계약월: 650
전세가: 650
전세 시세 기준일: 224
전세 시세 하한가: 224
전세 시세 상한가: 224


In [110]:
# Save raw data
with open('../data/eunma_real_estate.pickle', 'wb') as f:
    pickle.dump(eunma_real_estate, f, pickle.HIGHEST_PROTOCOL)

In [111]:
# load
with open('../data/eunma_real_estate.pickle', 'rb') as f:
    eunma_real_estate_loaded = pickle.load(f)

eunma_real_estate_loaded

{'101㎡': {'매매': {'date_real': ['2023.08.',
    '2023.08.',
    '2023.07.',
    '2023.07.',
    '2023.07.',
    '2023.07.',
    '2023.06.',
    '2023.06.',
    '2023.06.',
    '2023.06.',
    '2023.06.',
    '2023.06.',
    '2023.06.',
    '2023.06.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.05.',
    '2023.04.',
    '2023.04.',
    '2023.04.',
    '2023.04.',
    '2023.04.',
    '2023.04.',
    '2023.04.',
    '2023.03.',
    '2023.03.',
    '2023.03.',
    '2023.03.',
    '2023.03.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.02.',
    '2023.01.',
    '2023.01.',
    '2023.01.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.12.',
    '2022.11.',
    '2022.11.